In [1]:
import numpy as np
from scipy import stats
import pandas as pd

src = r"C:\Users\bb1755\Downloads\eq_corr_cont_rt_20210215.csv"
df = pd.read_csv(src, header=None)

In [2]:
mcorr = df.to_numpy(copy=True)
#mcorr = np.nan_to_num(mcorr)
#print(mcorr.shape)
mask_col = np.isnan(mcorr).any(axis=1)
mask_row = np.isnan(mcorr).any(axis=0)
mcorr = mcorr[~mask_row][:, ~mask_col].copy()
view = mcorr[:, :].copy()

In [3]:
def cov_to_corr(cov):
    vols = np.diag(cov)**0.5
    return np.diag(1/vols) @  cov @ np.diag(1/vols)

def calc_stats(cov, truecov, label):
    cond = np.linalg.cond(cov)
    fnorm = np.linalg.norm(cov - truecov, ord='fro')
    print(f'{label} cond {cond} norm {fnorm}')

In [4]:
import numpy as np,numpy.linalg

def _getAplus(A):
    eigval, eigvec = np.linalg.eig(A)
    Q = np.matrix(eigvec)
    xdiag = np.matrix(np.diag(np.maximum(eigval, 0)))
    return Q*xdiag*Q.T

def _getPs(A, W=None):
    W05 = np.matrix(W**.5)
    return  W05.I * _getAplus(W05 * A * W05) * W05.I

def _getPu(A, W=None):
    Aret = np.array(A.copy())
    Aret[W > 0] = np.array(W)[W > 0]
    return np.matrix(Aret)

def nearPD(A, nit=10):
    n = A.shape[0]
    W = np.identity(n) 
# W is the matrix used for the norm (assumed to be Identity matrix here)
# the algorithm should work for any diagonal W
    deltaS = 0
    Yk = A.copy()
    for k in range(nit):
        Rk = Yk - deltaS
        Xk = _getPs(Rk, W=W)
        deltaS = Xk - Rk
        Yk = _getPu(Xk, W=W)
    return Yk

In [5]:
calc_stats(view, 0, 'base')

base cond 4526573487.704492 norm 877.1612206309508


In [6]:
mcorr_near = nearPD(view)
calc_stats(mcorr_near, view, 'nearPD')

nearPD cond 1002092135.0792513 norm 0.06524672205573835


In [7]:
from nearest_correlation import nearcorr

mcorr_near2 = nearcorr(view, except_on_too_many_iterations=False)
calc_stats(mcorr_near2, view, 'nearPD2')

nearPD2 cond 1103166786938163.4 norm 0.06696968637544393


In [8]:
from sklearn.covariance import LedoitWolf
lw_corr =  LedoitWolf().fit(view).covariance_
lw_corr = cov_to_corr(lw_corr)
calc_stats(lw_corr, view, 'LW')

LW cond 2151642.651608075 norm 795.332393608271


In [9]:
from src.estimation.cov_cleaning import mp_denoise, RIE
mp_cov = mp_denoise(view, view.shape[0], 6*30)
mp_corr = cov_to_corr(mp_cov)
calc_stats(mp_corr, view, 'MP')

MP cond 4190.858009735371 norm 562.3551198525003


In [10]:
comp = lambda row,col : print(mp_corr[row,col], view[row,col])
#comp(0, 8)